In [1]:
import pickle
import numpy as np


PICKLE_FILE = '../embeds/v0.1_8B/persona_embeddings_Qwen.Qwen3-Embedding-8B_0-499.pkl'
with open(PICKLE_FILE, 'rb') as f:
    test_sentence1 = pickle.load(f)
print("Successfully loaded 'test_sentence' from file.")


PICKLE_FILE = '../embeds/v0.1_8B/persona_embeddings_Qwen.Qwen3-Embedding-8B_500-999.pkl'
with open(PICKLE_FILE, 'rb') as f:
    test_sentence2 = pickle.load(f)
print("Successfully loaded 'test_sentence' from file.")


PICKLE_FILE = '../embeds/v0.1_8B/persona_embeddings_Qwen.Qwen3-Embedding-8B_1000-1499.pkl'
with open(PICKLE_FILE, 'rb') as f:
    test_sentence3 = pickle.load(f)
print("Successfully loaded 'test_sentence' from file.")


PICKLE_FILE = '../embeds/v0.1_8B/persona_embeddings_Qwen.Qwen3-Embedding-8B_1500-1999.pkl'
with open(PICKLE_FILE, 'rb') as f:
    test_sentence4 = pickle.load(f)
print("Successfully loaded 'test_sentence' from file.")


PICKLE_FILE = '../embeds/v0.1_8B/persona_embeddings_Qwen.Qwen3-Embedding-8B_2000-2499.pkl'
with open(PICKLE_FILE, 'rb') as f:
    test_sentence5 = pickle.load(f)
print("Successfully loaded 'test_sentence' from file.")

PICKLE_FILE = '../embeds/v0.1_8B/persona_embeddings_Qwen.Qwen3-Embedding-8B_2500-2999.pkl'
with open(PICKLE_FILE, 'rb') as f:
    test_sentence6 = pickle.load(f)
print("Successfully loaded 'test_sentence' from file.")



hobbies_embeddings_dir="../initial_expedition/canonical_embeddings_qwen8B_v0.1.npy"

hobbies_embeddings = np.load(hobbies_embeddings_dir, allow_pickle=True)
print("Successfully loaded 'hobbies_embeddings' from file.")


Successfully loaded 'test_sentence' from file.
Successfully loaded 'test_sentence' from file.
Successfully loaded 'test_sentence' from file.
Successfully loaded 'test_sentence' from file.
Successfully loaded 'test_sentence' from file.
Successfully loaded 'test_sentence' from file.
Successfully loaded 'hobbies_embeddings' from file.


In [2]:
import pandas as pd

X=pd.read_csv("./final_dataset.csv")
X=X["0"]

In [4]:
from sentence_transformers import CrossEncoder, util
import numpy as np
from prediction_MatrixKNN import PredictionMatrixKNN
import os

# Define paths relative to this script
current_dir = "./"

# Paths derived from matrixKNN.ipynb structure
embeddings_path = os.path.join(current_dir, "../initial_expedition/canonical_embeddings_qwen8B_v0.1.npy")
csv_path = os.path.join(current_dir, "../initial_expedition/semantically_merged_hobbies_v0.1.csv")


# 3. Load the Accurate Model (Cross-Encoder) for Re-ranking
# This model is trained to judge the relevance of a (Query, Document) pair
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')


# --- THE PIPELINE ---

def recommend_hobbies(persona_text, persona_vec, k):

    matrixKNN = PredictionMatrixKNN(embeddings_path, csv_path)
    
    # === PHASE 1: RETRIEVAL (The "Fast" Step) ===
    # We cast a wide net to find the top 10 potential matches using cosine similarity

    # Semantic search returns a list of hobbies like ["football", "soccer", "basketball", ...]
    hits = matrixKNN.predict(persona_vec, k=10)[0]


    model_inputs = [[str(persona_text), str(hit)] for hit in hits]


    # Predict scores (returns a list of raw relevance scores, usually logits)
    cross_scores = cross_encoder.predict(model_inputs)
    
    # Attach new scores to our hits
    multihits=[]
    for i in range(len(hits)):
        multihits.append({'hobby_name': str(hits[i]), 'cross_score': cross_scores[i]})
        
    # Sort by the Cross-Encoder score (Descending)
    multihits = sorted(multihits, key=lambda x: x['cross_score'], reverse=True)
    
    # Return top 3
    final_recs = multihits[:k]
    return final_recs

# --- TEST RUN ---

# A tricky persona: Likes excitement but hates team sports. 
# A simple vector search often mistakenly recommends "Soccer" because of the word "sports".
persona_text = X[23]
persona_vec=test_sentence1[23]
recommendations = recommend_hobbies(persona_text, persona_vec, 5)

Loading embeddings from ./../initial_expedition/canonical_embeddings_qwen8B_v0.1.npy...
Loading hobby list from ./../initial_expedition/semantically_merged_hobbies_v0.1.csv...
Initializing MatrixKNN...
Initialization complete.


In [6]:
recommendations

[{'hobby_name': 'LEGO building (future cities, mythic castles)',
  'cross_score': -6.4832234},
 {'hobby_name': 'Nature walks and sketching', 'cross_score': -6.8122716},
 {'hobby_name': 'drawing/painting dinosaurs and robots',
  'cross_score': -7.053084},
 {'hobby_name': 'Sketching and abstract drawing', 'cross_score': -7.0933013},
 {'hobby_name': 'Doodling and sketching', 'cross_score': -7.125409}]

In [7]:
from prediction_CrossEncoder import PredictionCrossEncoder

ce=PredictionCrossEncoder(embeddings_path, csv_path)
p=ce.predict(persona_vec, persona_text=persona_text, k=5)
p

Loading embeddings from ./../initial_expedition/canonical_embeddings_qwen8B_v0.1.npy...
Loading hobby list from ./../initial_expedition/semantically_merged_hobbies_v0.1.csv...
Initializing MatrixKNN...
Initialization complete.
Loading Cross-Encoder model: cross-encoder/ms-marco-MiniLM-L-6-v2...
Cross-Encoder initialization complete.


[['LEGO building (future cities, mythic castles)',
  'Nature walks and sketching',
  'drawing/painting dinosaurs and robots',
  'Sketching and abstract drawing',
  'Doodling and sketching']]